# Análise de reclamações

Autor: Gabriel Bhering

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Os dados são provenientes do [kaggle](https://www.kaggle.com/gerosa/procon), e um [script](https://gist.github.com/bhering/d21097f5b125d65fd8c422d81695d7f7) foi rodado para higieniza-los. O que o script faz é remover algumas incoerencias (como traços no campo de CEP), e apagar campos sem dados que estavam preenchidos com `NULL` ou `Nao se aplica`. Com isso, a biblioteca consegue lidar melhor com os dados brutos. Abaixo estão algumas configurações para otimizar a leitura dos arquivos.

In [2]:
dtypes={
    'AnoCalendario': 'float',
    'DataArquivamento': 'str',
    'DataAbertura': 'str',
    'CodigoRegiao': 'float',
    'Regiao': 'str',
    'UF': 'str',
    'strRazaoSocial': 'str',
    'strNomeFantasia': 'str',
    'Tipo': 'float',
    'NumeroCNPJ': 'float',
    'RadicalCNPJ': 'float',
    'RazaoSocialRFB': 'str',
    'NomeFantasiaRFB': 'str',
    'CNAEPrincipal': 'float',
    'DescCNAEPrincipal': 'str',
    'Atendida': 'str',
    'CodigoAssunto': 'float',
    'DescricaoAssunto': 'str',
    'CodigoProblema': 'float',
    'DescricaoProblema': 'str',
    'SexoConsumidor': 'str',
    'FaixaEtariaConsumidor': 'str',
    'CEPConsumidor': 'float',
}
parse_dates=['DataArquivamento', 'DataAbertura']

Todos os .csv da pasta são lidos e compilados em um único dataframe `df`...

In [12]:
dataset_folder='./datasets/costumer_complaints'
all_files=glob.glob(os.path.join(dataset_folder,"*.csv"))
df=pd.concat((pd.read_csv(f, dtype=dtypes, parse_dates=parse_dates) for f in all_files))

In [13]:
df.head(10)

,AnoCalendario,DataArquivamento,DataAbertura,CodigoRegiao,Regiao,UF,strRazaoSocial,strNomeFantasia,Tipo,NumeroCNPJ,...,CNAEPrincipal,DescCNAEPrincipal,Atendida,CodigoAssunto,DescricaoAssunto,CodigoProblema,DescricaoProblema,SexoConsumidor,FaixaEtariaConsumidor,CEPConsumidor
0,2012.0,2012-07-13 17:37:19,2009-09-17 10:02:24,3.0,Sudeste,ES,SAMSUNG ELETRONICA DA AMAZONIA LTDA,SAMSUNG,1.0,2.802730e+11,...,2640000.0,"FABRICAÇÃO DE APARELHOS DE RECEPÇÃO, REPRODUÇÃ...",N,101.0,"Telefone ( Convencional, Celular, Interfone, E...",102.0,"Garantia (Abrangência, cobertura, etc.)",M,entre 31 a 40 anos,29115360.0
1,2012.0,2012-12-10 09:05:09,2009-09-21 09:01:49,5.0,Centro-oeste,GO,AMAZON PC INDUSTRIA E COMERCIO DE MICROCOMPUTA...,AMAZON PC,1.0,1.614079e+12,...,2621300.0,FABRICAÇÃO DE EQUIPAMENTOS DE INFORMÁTICA,S,102.0,Microcomputador / Produtos de Informática,102.0,"Garantia (Abrangência, cobertura, etc.)",M,entre 21 a 30 anos,75911000.0
2,2012.0,2012-07-17 15:25:26,2009-09-22 09:49:56,3.0,Sudeste,ES,AON AFFINITY DO BRASIL SERVIÇOS E CORRETORA DE...,AON AFFINITY DO BRASIL,1.0,2.143320e+12,...,6629100.0,"ATIVIDADES AUXILIARES DOS SEGUROS, DA PREVIDÊN...",N,79.0,Seguro,54.0,Não pagamento de indenização (seguros em geral),M,entre 41 a 50 anos,29045060.0
3,2012.0,2012-07-17 14:47:52,2009-10-23 15:58:49,3.0,Sudeste,ES,BOMPRECO BAHIA SUPERMERCADOS LTDA,BOM PRECO,1.0,9.742262e+13,...,4711301.0,"COMÉRCIO VAREJISTA DE MERCADORIAS EM GERAL, CO...",N,102.0,Microcomputador / Produtos de Informática,102.0,"Garantia (Abrangência, cobertura, etc.)",F,entre 41 a 50 anos,29055280.0
4,2012.0,2012-12-05 13:50:18,2009-05-15 12:33:24,5.0,Centro-oeste,DF,REAL LEASING S.A. ARRENDAMENTO MERCANTIL,SUDAMERIS,1.0,4.719315e+13,...,6440900.0,ARRENDAMENTO MERCANTIL,N,63.0,Financeira,28.0,Cobrança indevida.,F,entre 51 a 60 anos,70238110.0
5,2012.0,2012-10-31 10:23:32,2010-02-11 16:35:27,5.0,Centro-oeste,GO,BANCO BMG,NaN,1.0,6.118668e+13,...,6422100.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",S,62.0,Empresa de Cobrança,28.0,Cobrança indevida.,M,entre 41 a 50 anos,NaN
6,2012.0,2012-10-09 14:47:31,2010-03-08 12:25:24,2.0,Nordeste,CE,LG ELECTRONICS DO BRASIL LTDA,NaN,1.0,1.166372e+12,...,2622100.0,FABRICAÇÃO DE PERIFÉRICOS PARA EQUIPAMENTOS DE...,S,101.0,"Telefone ( Convencional, Celular, Interfone, E...",113.0,Falta de peca de reposição,M,entre 31 a 40 anos,62960000.0
7,2012.0,2012-09-14 14:33:24,2005-08-11 10:16:23,5.0,Centro-oeste,GO,JUTAI 661 EQUIPAMENTOS ELETRONICOS LTDA (BENQ),SIEMENS CELULARES,1.0,7.560958e+12,...,2632900.0,FABRICAÇÃO DE APARELHOS TELEFÔNICOS E DE OUTRO...,N,101.0,"Telefone ( Convencional, Celular, Interfone, E...",113.0,Falta de peca de reposição,M,entre 21 a 30 anos,75250000.0
8,2012.0,2012-09-24 14:39:49,2006-11-07 09:07:41,5.0,Centro-oeste,GO,LG ELECTRONICS DO BRASIL LTDA,LG,1.0,1.166372e+12,...,2622100.0,FABRICAÇÃO DE PERIFÉRICOS PARA EQUIPAMENTOS DE...,S,96.0,Televisão / Vídeo Cassete / Filmadora / Video-...,102.0,"Garantia (Abrangência, cobertura, etc.)",M,entre 51 a 60 anos,75240000.0
9,2012.0,2012-04-13 16:39:42,2010-02-18 16:27:20,3.0,Sudeste,MG,MABE ITU ELETRODOMESTICOS S.A.,DAKO / GE,1.0,2.147737e+12,...,2751100.0,"FABRICAÇÃO DE FOGÕES, REFRIGERADORES E MÁQUINA...",S,93.0,Fogão e Microondas,105.0,Produto com vício,F,Nao Informada,NaN
